In [ ]:
!pip install altair vega_datasets

# Destilacion de Logits

In [1]:
import pandas as pd
import altair as alt
import lib.utils.viz.KD as KD

In [15]:
KD.omniplot(folder="./Cifar10/ResNet101/exp2/students")

alt.RepeatChart(...)

In [3]:




#data=KD.load_data(folder="./Cifar10/ResNet101/exp1/students")
KD.load_and_plot(phase='test',field="acc",center=True)


alt.Chart(...)

## Destilacion con features

In [1]:
import altair as alt
from lib.utils.viz import Feat
Feat.load_and_plot("./Cifar10/ResNet101/exp7/students")

alt.LayerChart(...)

In [11]:
#data=Feat.load_data("./Cifar10/ResNet101/exp7/students")



Feat.load_and_plot(phase="train",field="acc",center=True,scale="symlog")

alt.Chart(...)

In [13]:
def plot(data,phase,field, center=False, scale=None):
    detalle=['test_acc', 'test_teacher/acc', 'test_loss', 'test_eval',
           'train_acc', 'train_teacher/acc', 'train_loss', 'train_eval','distillation','last_layer','layer']
    
    
    if scale is None:
        scale='log' if field in ['loss','eval'] else 'linear'
    else:
        print("scale", scale)
    
    field_dict={'acc':"Accuracy [%]", 'eval':"Perdida Cross Entropy", 'loss':"Perdida de Destilación"
        
    }
    
    data=data[data['distillation']!='nst_gauss']
    #data['train_acc']-=data['test_acc']
    
    
    bar=alt.Chart(data).mark_point().encode(
        alt.X('layer:O', scale=alt.Scale(zero=False,base=10),title="Capa"),
        alt.Y('%s_%s'%(phase,field), 
              scale=alt.Scale(zero=False, type=scale), 
              title=field_dict[field]),
        color=alt.Color('student last_layer'if center else 'student', legend=alt.Legend(title="Modelo, KD" if center else "Modelo")),
        fill=alt.Color('student last_layer'if center else 'last_layer', legend=alt.Legend(title="Modelo, KD" if center else "Destilacion Logits")),
        shape=alt.Color('distillation', legend=alt.Legend(title="Destilacion feat.")),
        size=alt.value(50),
        tooltip=detalle
    ).interactive()
    
    if field=='acc':
        accs = {'Model':['MobileNet','ResNet18','ResNet101'],
                      'ce_train':[95.73,98.15,98.52],
                      'ce_test':[87.8,90.58,90.68]}
                                 
        
        
        df=pd.DataFrame(accs)
        
        if center:
            d=dict(list(zip(accs['Model'],accs['ce_%s'%phase])))
            data['%s_acc'%phase]-=[d[i] for i in data['student']]
        
        else:
            aggregates = alt.Chart(df).mark_rule(opacity=0.5).encode(
                        y='ce_%s:Q'%phase,
                        color='Model:N',
                        size=alt.value(2))

        
            return (aggregates+bar).properties(width=600,height=400) 
    return bar.properties(width=600,height=400)

import pandas as pd
data=Feat.load_data("./Cifar10/ResNet101/exp7/students")
plot(data,"test","acc",scale="symlog")

scale symlog


alt.LayerChart(...)

In [91]:
data.head()

,Unnamed: 0,Unnamed: 0.1,distillation,epoch,epochs,lambda,last_layer,layer,lr,pre,...,test_batch_size,test_eval,test_loss,test_teacher/acc,train_acc,train_batch_size,train_eval,train_loss,train_teacher/acc,student last_layer
0,0,0,PKT,90,100,1.0,KD,3,0.01,50,...,100,0.547186,0.003339,90.68,95.622,128,0.166026,0.001806,98.238,"MobileNet,KD"
1,1,1,PKT,95,100,1.0,KD,1,0.01,50,...,100,0.549994,0.003597,90.68,95.444,128,0.177137,0.001846,98.260,"MobileNet,KD"
2,2,2,PKT,92,100,1.0,KD,0,0.01,50,...,100,0.571599,0.003312,90.68,95.658,128,0.165046,0.001797,98.240,"MobileNet,KD"
3,3,3,PKT,81,100,1.0,KD,2,0.01,50,...,100,0.588225,0.003313,90.68,95.598,128,0.165880,0.001795,98.250,"MobileNet,KD"
4,4,4,hint,99,100,1.0,KD,3,0.01,50,...,100,0.672916,141351.086406,90.68,91.750,128,0.354643,105205.985794,98.238,"MobileNet,KD"


In [72]:
accs = {'Model':['MobileNet','ResNet18','ResNet101'],
                      'ce_train':[95.73,98.15,98.52],
                      'ce_test':[87.8,90.58,90.68]}

d=dict(list(zip(accs['Model'],accs['ce_%s'%"train"])))



0      -5.36
1      -5.32
2      -5.39
3      -5.56
4      -7.14
       ...  
99     -8.56
100   -20.75
101   -17.22
102   -43.35
103   -52.09
Name: test_acc, Length: 104, dtype: float64

# Dataset Merging

In [10]:
import pandas as pd
def load_KD_data(folder="./Cifar10/ResNet101/exp1/students"):
    data = pd.read_csv(folder+"/summary.csv") 
    data = data[data["student"] != 'EfficientNetB0']
    if "temp" not in data.columns:
        d_arr=[d.split(",")[0] for d in data["distillation"]]
        t_arr=[float(d.split(",T-")[1]) for d in data["distillation"]]
        data["distillation"]=d_arr
        data["temp"]=t_arr
    return data

drop_cols="test_teacher/acc train_teacher/acc lr	epochs	train_batch_size teacher	test_batch_size resume epoch".split()


d1=load_KD_data(folder="./Cifar10/ResNet101/exp1/students").drop(["test_teacher/acc","train_teacher/acc"],axis=1)
d2=load_KD_data(folder="./Cifar10/ResNet101/exp2/students").drop(drop_cols,axis=1)
d3=load_KD_data(folder="./Cifar10/ResNet101/exp3/students").drop(drop_cols,axis=1)
db=d3.merge(d2,on=['student','distillation','temp'],suffixes=('_R01','_R1')).drop(['Unnamed: 0_R01','Unnamed: 0_R1'],axis=1)
db=db.merge(d1,on=['student','distillation','temp']).drop(['epoch','Unnamed: 0'],axis=1)

In [11]:
detalle=['test_acc' + s for s in ["","_R1",'_R01']]

alt.Chart(db).mark_point().encode(
    alt.X(alt.repeat("column"), type='quantitative',scale=alt.Scale(zero=False,base=10,type='linear')),
    alt.Y(alt.repeat("row"), type='quantitative',scale=alt.Scale(zero=False,base=10,type='linear')),
    color='student',
    shape='distillation'
).properties(
    width=150,
    height=150
).repeat(
    row=detalle,
    column=detalle
)


alt.RepeatChart(...)

In [108]:
db.head()

,test_acc_R01,test_loss_R01,test_eval_R01,train_acc_R01,train_loss_R01,train_eval_R01,student,distillation,temp,test_acc_R1,...,test_eval_R1,train_acc_R1,train_loss_R1,train_eval_R1,test_acc,test_loss,test_eval,train_acc,train_loss,train_eval
0,89.33,2462.161829,0.492432,97.648,332.814397,0.066563,ResNet18,KD_CE,100.0,10.00,...,2.302861,9.840,1.151709e+04,2.303418,90.32,2467.345938,0.493469,98.152,286.173518,0.057235
1,90.02,0.187363,0.337151,98.002,0.058635,0.066467,ResNet18,KD_CE,1.0,10.00,...,2.326473,10.000,1.246259e+00,2.328405,90.28,0.251264,0.450828,98.360,0.032126,0.049576
2,89.34,243786.745078,0.487573,97.604,33371.804589,0.066744,ResNet18,KD_CE,1000.0,10.00,...,2.303176,9.934,1.152305e+06,2.304609,90.24,228784.270391,0.457569,98.288,26333.361176,0.052667
3,90.05,21.899282,0.437634,97.632,3.351010,0.066709,ResNet18,KD_CE,10.0,10.00,...,2.302697,9.946,1.151815e+02,2.303356,90.02,24.829716,0.496368,98.334,2.722346,0.054231
4,89.06,5.887921,0.469171,97.784,0.793954,0.061620,ResNet18,KD_CE,5.0,13.19,...,2.302721,9.984,2.882868e+01,2.303233,90.00,6.059729,0.483544,98.126,0.735777,0.057665


In [104]:
detalle

['test_acc', 'test_acc_R1', 'test_acc_R01']

In [ ]:
db[test_acc_R01	] test_acc_R01